In [15]:
import tensorflow as tf
import numpy as np

In [16]:
n_inputs = 3
n_neurons = 5
#                                ROWS  FEATURES 
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))

b = tf.Variable(tf.zeros([1, n_neurons]))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)

init = tf.global_variables_initializer()


X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) # t = 0
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) # t = 1


with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})
    
print(Y0_val)
print(Y1_val)

[[-0.4621275   0.99019426  0.99393827 -0.16810046 -0.9741581 ]
 [-0.89096606  0.99999994  0.9998044  -0.750714   -0.99577546]
 [-0.98208636  1.          0.9999937  -0.94463784 -0.9993157 ]
 [-0.4469795   0.99984026 -1.         -0.9999972   1.        ]]
[[-0.51039326  1.          0.99421346 -0.9937782  -0.95482427]
 [ 0.9450379   0.90510184  0.95794535 -0.21349257 -0.8384108 ]
 [ 0.38113478  1.          0.9940177  -0.9078136  -0.6877402 ]
 [ 0.29393965  0.9565572   0.9605589   0.55280244  0.3217394 ]]


In [20]:
tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) # t = 0
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) # t = 1


# NEW
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, 
                                                [X0, X1], 
                                                dtype=tf.float32)
Y0, Y1 = output_seqs
#

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_res, Y1_res = sess.run([Y0, Y1], feed_dict={X0:X0_batch, X1:X1_batch})

print(Y0_res,'\n', Y1_res)
    

[[ 0.27309662  0.27397233 -0.03838524 -0.34390745 -0.337415  ]
 [-0.34945273 -0.60092264 -0.2456865  -0.98069733  0.5609472 ]
 [-0.76569897 -0.9315958  -0.432713   -0.999611    0.92456627]
 [ 0.07165036 -0.99861604  0.13814802 -0.93735474  0.8447625 ]] 
 [[-0.98769724 -0.99783695 -0.6756848  -0.9999431   0.9972622 ]
 [-0.4179167   0.39433694  0.8251724   0.20969756  0.357178  ]
 [-0.9680483  -0.9226516   0.87026864 -0.9979022   0.9914147 ]
 [-0.5943253  -0.55875236  0.7661928  -0.9214663   0.9641249 ]]


In [39]:
tf.reset_default_graph()

n_neurons = 5
n_inputs = 3
n_steps = 2  

#new
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, 
                                                 X_seqs, #earlier [X0, X1]... 
                                                 dtype=tf.float32)

output = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

init = tf.global_variables_initializer()

# X_batch = np.array([
#         # t = 0      t = 1 
#         [[0, 1, 2], [9, 8, 7]], # próbka 1
#         [[3, 4, 5], [0, 0, 0]], # próbka 2
#         [[6, 7, 8], [6, 5, 4]], # próbka 3
#         [[9, 0, 1], [3, 2, 1]], # próbka 4
#     ])

# sequences have to be same length - pad to the longest in time with zeros
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # próbka 1
        [[3, 4, 5], [0, 0, 0]], # próbka 2
        [[6, 7, 8], [6, 5, 4]], # próbka 3
        [[9, 0, 1], [3, 2, 1]], # próbka 4
    ])



with tf.Session() as sess:
    sess.run(init)
    outputs_val = sess.run(output, feed_dict={X:X_batch})

print(outputs_val)

[[[-0.21533237  0.8727285  -0.61737883  0.24111131 -0.73497665]
  [-0.22908157  0.99995905 -0.97887325  0.30840403  0.9422193 ]]

 [[-0.29368347  0.99891543 -0.95577234  0.48242706 -0.6401998 ]
  [-0.06015785 -0.4573911   0.5481148  -0.37309408  0.37851453]]

 [[-0.3682777   0.99999136 -0.99568594  0.66756177 -0.5208888 ]
  [ 0.11694624  0.9958281  -0.79027736 -0.12894602  0.9455721 ]]

 [[ 0.99893355 -0.78009     0.7999048   0.9059533   0.99979305]
  [ 0.72375476  0.93463176 -0.95335525  0.40686134  0.80762357]]]


In [33]:
tf.reset_default_graph()

n_neurons = 5
n_inputs = 3
n_steps = 2  

#new
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])


basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, 
                                                 X, #earlier unstacked X 
                                                 dtype=tf.float32,
                                                 swap_memory=True)


init = tf.global_variables_initializer()


X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # próbka 1
        [[3, 4, 5], [0, 0, 0]], # próbka 2
        [[6, 7, 8], [6, 5, 4]], # próbka 3
        [[9, 0, 1], [3, 2, 1]], # próbka 4
    ])



with tf.Session() as sess:
    sess.run(init)
    outputs_val = sess.run(outputs, feed_dict={X:X_batch})

print(outputs_val)

[[[-0.71023595  0.07329487  0.8815821  -0.33744517  0.00550849]
  [-0.60659313  0.85433555  0.9604514   0.9999663  -0.7668698 ]]

 [[-0.8766344   0.36907664  0.9859492   0.88636905 -0.3611181 ]
  [ 0.5608052   0.19889209 -0.11843139  0.59844434 -0.31479818]]

 [[-0.950257    0.60518014  0.9984105   0.9964113  -0.6421658 ]
  [ 0.6128228   0.7323126   0.64111805  0.99981105 -0.8779617 ]]

 [[ 0.9296684   0.39289638 -0.9984451   0.9996214  -0.9945635 ]
  [ 0.86451554  0.08912306 -0.3203876   0.9808753  -0.84579855]]]


In [46]:
tf.reset_default_graph()

n_neurons = 5
n_inputs = 3
n_steps = 2  

#new
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
seq_len = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, 
                                   X, #earlier [X0, X1]... 
                                   dtype=tf.float32,
                                   sequence_length = seq_len,
                                   swap_memory=True)



init = tf.global_variables_initializer()

# sequences have to be same length - pad to the longest in time with zeros
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # próbka 1
        [[3, 4, 5], [0, 0, 0]], # próbka 2
        [[6, 7, 8], [6, 5, 4]], # próbka 3
        [[9, 0, 1], [3, 2, 1]], # próbka 4
    ])
seq_len_batch = [2,1,2,2]


with tf.Session() as sess:
    sess.run(init)
    outputs_val = sess.run(outputs, feed_dict={X:X_batch, seq_len:seq_len_batch})

print(outputs_val)

[[[-0.4468847   0.73098195  0.6184553   0.82631767 -0.80680156]
  [-0.53638566  0.9999983  -0.9864757   0.9999985  -0.9999664 ]]

 [[-0.4972186   0.9986192   0.36040422  0.99935436 -0.9959174 ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.5444144   0.99999386  0.03218993  0.9999978  -0.99992174]
  [-0.36950937  0.9997204  -0.98375636  0.9993262  -0.998807  ]]

 [[-0.92707425  0.99262714 -0.9997556   0.96766424 -0.6476023 ]
  [ 0.26296636  0.96137667 -0.983384    0.71805733 -0.95108473]]]


In [51]:
#klasyfikator sekwencji
tf.reset_default_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
learning_rate = 0.001


# Construction phase

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                         logits = logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy =tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()


Instructions for updating:
Use keras.layers.dense instead.


In [53]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('/tmp/data/')
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [57]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape(-1, n_steps, n_inputs)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
        acc_test = accuracy.eval(feed_dict={X:X_test, y:y_test})
        print(f'epoch: {epoch} | train accuracy: {acc_train:.4f} | test accuracy: {acc_test:.4f}')

epoch: 0 | train accuracy: 0.9400 | test accuracy: 0.9225
epoch: 1 | train accuracy: 0.9533 | test accuracy: 0.9422
epoch: 2 | train accuracy: 0.9733 | test accuracy: 0.9545
epoch: 3 | train accuracy: 0.9533 | test accuracy: 0.9589
epoch: 4 | train accuracy: 0.9800 | test accuracy: 0.9618
epoch: 5 | train accuracy: 0.9733 | test accuracy: 0.9727
epoch: 6 | train accuracy: 0.9933 | test accuracy: 0.9695
epoch: 7 | train accuracy: 0.9667 | test accuracy: 0.9681
epoch: 8 | train accuracy: 0.9667 | test accuracy: 0.9692
epoch: 9 | train accuracy: 0.9667 | test accuracy: 0.9661
epoch: 10 | train accuracy: 0.9800 | test accuracy: 0.9726
epoch: 11 | train accuracy: 1.0000 | test accuracy: 0.9758
epoch: 12 | train accuracy: 0.9667 | test accuracy: 0.9708
epoch: 13 | train accuracy: 0.9867 | test accuracy: 0.9738
epoch: 14 | train accuracy: 0.9800 | test accuracy: 0.9710
epoch: 15 | train accuracy: 0.9867 | test accuracy: 0.9728
epoch: 16 | train accuracy: 0.9867 | test accuracy: 0.9767
epoch: 

In [89]:
# time series forecasting

tf.reset_default_graph()

n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1
learning_rate = 0.001


X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
cell = tf.contrib.rnn.OutputProjectionWrapper(cell, output_size=n_outputs)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

loss = tf.reduce_mean(tf.square(outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()


In [90]:
import matplotlib.pyplot as plt

t_min, t_max = 0, 30
resolution = 0.1

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
    Ts = t0 + np.arange(0., n_steps + 1) * resolution
    ys = time_series(Ts)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)



In [91]:

n_iterations = 1500
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
        sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X:X_batch, y:y_batch})
            print(f'Epoch: {iteration} | MSE: {mse}')


Epoch: 0 | MSE: 17.02533531188965
Epoch: 100 | MSE: 0.7531074285507202
Epoch: 200 | MSE: 0.3197920322418213
Epoch: 300 | MSE: 0.13721634447574615
Epoch: 400 | MSE: 0.05726533383131027
Epoch: 500 | MSE: 0.05333767831325531
Epoch: 600 | MSE: 0.06567077338695526
Epoch: 700 | MSE: 0.056353483349084854
Epoch: 800 | MSE: 0.05108434706926346
Epoch: 900 | MSE: 0.062102217227220535
Epoch: 1000 | MSE: 0.050589025020599365
Epoch: 1100 | MSE: 0.05000985041260719
Epoch: 1200 | MSE: 0.03968095779418945
Epoch: 1300 | MSE: 0.05358519032597542
Epoch: 1400 | MSE: 0.0451069101691246


In [92]:
with tf.Session() as sess:
    sequence = [0.]*n_steps
    for iteration in range(300):
        X_batch = np.array(sequence[-n_steps:]).reshape(1, n_steps, 1)
        y_pred = sess.run(outputs, feed_dict={X:X_batch})
        sequence.append(y_pred[0, -1, 0])

FailedPreconditionError: Attempting to use uninitialized value rnn/output_projection_wrapper/bias
	 [[node rnn/output_projection_wrapper/bias/read (defined at <ipython-input-89-fe0f437cc1a2>:18) ]]
	 [[node rnn/transpose_1 (defined at <ipython-input-89-fe0f437cc1a2>:18) ]]

Caused by op 'rnn/output_projection_wrapper/bias/read', defined at:
  File "/home/tom/anaconda3/envs/tf/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/tom/anaconda3/envs/tf/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tom/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tom/anaconda3/envs/tf/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/tom/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/ho

In [83]:
sequence

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [85]:
X_batch

array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]])